In [1]:
### Imports

import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

In [2]:
input_folder = '../archive/'

In [3]:
### @author selfadri
### Load the datasets into pandas dataframes from csv

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
datasets = {}
for dirname, _, filenames in os.walk(input_folder):
    for filename in filenames:
        if filename.split('.')[-1] != 'csv':
            continue
        datasets['_'.join((dirname.split('postings')[-1].replace('/',''),filename.split('.')[-2])).strip(input_folder).lstrip('_')] = pd.read_csv(os.path.join(dirname, filename))

pprint([*datasets.keys()])

['job_postings',
 'maps_skills',
 'maps_industries',
 'ompany_details_company_industries',
 'ompany_details_company_specialities',
 'ompany_details_companies',
 'ompany_details_employee_counts',
 'job_details_benefits',
 'job_details_salaries',
 'job_details_job_industries',
 'job_details_job_skills']


In [35]:
### @author selfadri

# Calculate labels, which indicate salary, from any available min-med-max values
untrimmed_y = np.mean(datasets['job_postings'].iloc[:,4:7], axis=1)

# Clean dataset to remove unwanted samples
cond = untrimmed_y.notna() & (datasets['job_postings']['pay_period'] != "ONCE")
y = untrimmed_y[cond]
X = datasets['job_postings'][cond]

# Adjust for the `pay_period`
assert np.all(np.unique(X.iloc[:,7]) == ['HOURLY', 'MONTHLY', 'WEEKLY', 'YEARLY'])
y[X['pay_period'] == "YEARLY"] *= 1
y[X['pay_period'] == "MONTHLY"] *= 12
y[X['pay_period'] == "WEEKLY"] *= 50
y[X['pay_period'] == "HOURLY"] *= 40 * 50
X

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",...,NaN,Entry level,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101
3,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,...,NaN,Mid-Senior level,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346
8,3757936097,18213359.0,Dishwasher,"descriptionTitle\n\n $2,000 Sign-on Bonus Guar...",NaN,19.30,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699089324
11,3757935381,19181907.0,Insights Analyst - Auto Industry,Who We Are\n\nEscalent is an award-winning dat...,64000.0,NaN,58000.0,YEARLY,Full-time,United States,...,1.700000e+12,Entry level,NaN,1.699090e+12,escalent.hrmdirect.com,0,FULL_TIME,USD,BASE_SALARY,1699134111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33231,1198636648,NaN,Mental Health Practitioners,Gail M. Yost and Associates is hiring full tim...,NaN,100000.00,NaN,YEARLY,Full-time,"Minneapolis, MN",...,NaN,NaN,NaN,1.699050e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,1699063051
33233,1029078768,61469.0,Registered Nurse (RN) Vaccinator,United Staffing Solutions is partnering with o...,50.0,NaN,50.0,HOURLY,Part-time,"Muskegon, MI",...,NaN,NaN,NaN,1.692740e+12,NaN,0,PART_TIME,USD,BASE_SALARY,1
33236,903408693,3894635.0,Office Associate,Provide clerical and administrative support to...,42000.0,NaN,37000.0,YEARLY,Full-time,"Albany, GA",...,NaN,NaN,NaN,1.692730e+12,NaN,1,FULL_TIME,USD,BASE_SALARY,1
33244,85008768,NaN,Licensed Insurance Agent,While many industries were hurt by the last fe...,52000.0,NaN,45760.0,YEARLY,Full-time,"Chico, CA",...,NaN,NaN,NaN,1.692750e+12,NaN,1,FULL_TIME,USD,BASE_SALARY,1


In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13351 entries, 0 to 33245
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   job_id                      13351 non-null  int64  
 1   company_id                  13129 non-null  float64
 2   title                       13351 non-null  object 
 3   description                 13351 non-null  object 
 4   max_salary                  11110 non-null  float64
 5   med_salary                  2241 non-null   float64
 6   min_salary                  11110 non-null  float64
 7   pay_period                  13351 non-null  object 
 8   formatted_work_type         13351 non-null  object 
 9   location                    13351 non-null  object 
 10  applies                     7372 non-null   float64
 11  original_listed_time        13351 non-null  float64
 12  remote_allowed              2098 non-null   float64
 13  views                       10950 no

In [37]:
### Some samples do not have the salary data needed for our labels.
print(f'{(untrimmed_y.size - y.size) / untrimmed_y.size * 100 :.2f} % of samples are lost due to not having salary :)')
print(f'{y.size} samples remain.')
print('Unfortunately, this is normal for LinkedIn.')
print('Maybe we could make a binary classifier for whether or not the employer would post the salary :)')

59.84 % of samples are lost due to not having salary :)
13351 samples remain.
Unfortunately, this is normal for LinkedIn.
Maybe we could make a binary classifier for whether or not the employer would post the salary :)


In [38]:
### @author selfadri

# Just USD
print(np.unique(datasets['job_details_salaries']['currency']))

# Not here, nothing new to learn from the job_details_salaries csv
print(datasets['job_details_salaries'][datasets['job_details_salaries']['job_id'] == datasets['job_postings'].iloc[2]['job_id']])

# Here's the benefit types recorded
pprint([*np.unique(np.asarray(datasets['job_details_benefits']['type'], str))])

# Here's the work types
pprint([*np.unique(X['work_type'])])

# Here's the titles
print(pd.DataFrame(np.unique(X['title'])))
pprint(str(Counter(X['title']))[:1000])

# Here's the locations
pprint(str(Counter(X['location']))[:1000])

# Only a few job postings have skill description paragraphs
print(y.size - Counter(np.array(X['skills_desc'] ,str))['nan'])

# There are a handful of job types, plenty of each category
print(Counter(datasets['job_details_job_skills'].iloc[:,1]))

['USD']
Empty DataFrame
Columns: [salary_id, job_id, max_salary, med_salary, min_salary, pay_period, currency, compensation_type]
Index: []
['401(k)',
 'Child care support',
 'Commuter benefits',
 'Dental insurance',
 'Disability insurance',
 'Medical insurance',
 'Paid maternity leave',
 'Paid paternity leave',
 'Pension plan',
 'Student loan assistance',
 'Tuition assistance',
 'Vision insurance']
['CONTRACT', 'FULL_TIME', 'INTERNSHIP', 'OTHER', 'PART_TIME', 'TEMPORARY']
                                                      0
0                   Research Associate II, Pathobiology
1                                      Account Manager 
2      Airport Senior Project Manager – Large Termin...
3                               Applications Developer 
4         Associate Director Marketing Asset Management
...                                                 ...
9414  in-house Contracting Counsel (junior-mid level...
9415                 oracle SCM TECHNO CLOUD FUNCTIONAL
9416              

### Features
We will consider 10 features for now.
- Experience Level
- Job Title (Later, embed this)
- Work Type (full time, part time, intern, etc.)
- Location
- Skills
- Job Industry
- Company Industry
- Company Employee Count
- Benefits
- Company LinkedIn Follower Count
- Remote Work Allowed

### Scope
The location is always in the United States, and the currency is always measured in USD.

### Labels (Salary)
We extracted our labels

In [138]:
X_trim=X.copy()
X_trim=X_trim.drop(columns=['job_id', 'title','description', 'max_salary', 'med_salary',
                  'min_salary', 'pay_period', 'applies', 'original_listed_time',
                  'views', 'job_posting_url', 'application_url', 'application_type',
                  'expiry', 'closed_time', 'listed_time','posting_domain', 'sponsored', 'currency',
                   'compensation_type', 'scraped', 'skills_desc', 'formatted_work_type', 'company_id'])
# fill na of 'remote_allowed' with False
X_trim['remote_allowed'].fillna(0, inplace=True)
X_trim['remote_allowed'] = X_trim['remote_allowed'].astype(int)
X_trim['work_type'].fillna("Unknown", inplace=True)
X_trim['location'].fillna("Unknown", inplace=True)
# work type full time = 1, 0 else
X_trim['work_type'] = (X_trim['work_type'] == 'FULL_TIME').astype(int)
X_trim['formatted_experience_level'].fillna("Unknown", inplace=True)
#X_trim['company_id'].fillna(0, inplace=True)
#X_trim['company_id'] = X_trim['company_id'].astype(float)
# take state form location]
X_trim['location'] = X_trim['location'].str.split(',').str[-1].str.strip().str.upper()
X_trim

,location,remote_allowed,formatted_experience_level,work_type
0,SC,0,Entry level,1
3,CA,0,Entry level,1
4,UNITED STATES,1,Mid-Senior level,1
8,CA,0,Entry level,1
11,UNITED STATES,1,Entry level,1
...,...,...,...,...
33231,MN,0,Unknown,1
33233,MI,0,Unknown,0
33236,GA,0,Unknown,1
33244,CA,0,Unknown,1


In [139]:
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [140]:
enc = OneHotEncoder(sparse_output=False)
enc.fit(X_trim[['location', 'formatted_experience_level']])

OneHotEncoder(sparse_output=False)

In [141]:
# one hot encode

X_ohe = enc.transform(X_trim[['location', 'formatted_experience_level']])
# replace the original columns with the one hot encoded columns
X_trimmed = X_trim.copy()
X_trimmed = X_trimmed.drop(columns=['location', 'formatted_experience_level'])
feature_names = []
for i in range(len(enc.categories_)):
    for j in enc.categories_[i]:
        feature_names.append(f'feature_{i}_{j}')
X_ohe = pd.DataFrame(X_ohe, columns=feature_names).reset_index(drop=True)
X_trimmed = X_trimmed.reset_index(drop=True)
# add the one hot encoded columns to the dataframe
X_trimmed = pd.concat([X_trimmed, X_ohe], axis=1)
X_trimmed

,remote_allowed,work_type,feature_0_AK,feature_0_AL,feature_0_ALABAMA AREA,feature_0_ALBUQUERQUE-SANTA FE METROPOLITAN AREA,feature_0_AR,feature_0_ATLANTA METROPOLITAN AREA,feature_0_AZ,feature_0_AZ AREA,...,feature_0_WISCONSIN METROPOLITAN AREA,feature_0_WV,feature_0_WY,feature_1_Associate,feature_1_Director,feature_1_Entry level,feature_1_Executive,feature_1_Internship,feature_1_Mid-Senior level,feature_1_Unknown
0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13346,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13347,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13348,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13349,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [142]:
X_train, X_test, y_train, y_test = train_test_split(X_trimmed, y, test_size=0.2, random_state=42)

In [143]:
# datafrme to numpy array
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [144]:
X_train[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0.])

In [145]:
svr = svm.SVR()
svr.fit(X_train[:500], y_train[:500])
svr.score(X_test, y_test)


-0.0010492509485313661

In [146]:
svr.predict(X_train[:5]), y_train[:5]

(array([75017.70305573, 75015.73559677, 74999.42710591, 75007.48681158,
        75018.59515761]),
 array([100325. ,  90000. ,  40000. , 162180.5,  95500. ]))